# PDF MAKER

This notebook compiles the printable pdf version of the game!

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from pandas import read_csv
from pathlib import Path

In [ ]:
all_pages = []
front_pages = []
back_pages = []

def setup_pdf():
    fig = plt.figure(dpi=300)
    plt.xlim(0,8.5)
    plt.ylim(0,11)
    plt.axis('off')
    # Standard 8.5x11 paper
    fig.set_size_inches((8.5, 11))

    # Guide lines for cutting
    card_width = 2.5
    buffer1 = (8.5 - 2.5*3)/4
    for v in [buffer1, buffer1+card_width, 
            2*buffer1+card_width, 2*buffer1+2*card_width,
            3*buffer1+2*card_width,3*buffer1+3*card_width]:
        plt.axvline(v, c='k', linestyle=':',lw=1)

    card_height = 3.5
    buffer2 = (11 - 3.5*3)/4
    for v in [buffer2, buffer2+card_height, 
            2*buffer2+card_height, 2*buffer2+2*card_height,
            3*buffer2+2*card_height,3*buffer2+3*card_height]:
        plt.axhline(v, c='k', linestyle=':',lw=1)

    # Define extents
    e1 = [buffer1, buffer1+card_width, 3*buffer2+2*card_height,3*buffer2+3*card_height]
    e2 = [2*buffer1+card_width, 2*buffer1+2*card_width, 3*buffer2+2*card_height,3*buffer2+3*card_height]
    e3 = [3*buffer1+2*card_width,3*buffer1+3*card_width, 3*buffer2+2*card_height, 3*buffer2+3*card_height]

    e4 = [buffer1, buffer1+card_width, 2*buffer2+card_height, 2*buffer2+2*card_height]
    e5 = [2*buffer1+card_width, 2*buffer1+2*card_width, 2*buffer2+card_height, 2*buffer2+2*card_height]
    e6 = [3*buffer1+2*card_width,3*buffer1+3*card_width, 2*buffer2+card_height, 2*buffer2+2*card_height]

    e7 = [buffer1, buffer1+card_width, buffer2, buffer2+card_height]
    e8 = [2*buffer1+card_width, 2*buffer1+2*card_width, buffer2, buffer2+card_height]
    e9 = [3*buffer1+2*card_width,3*buffer1+3*card_width, buffer2, buffer2+card_height]

    return fig, [e1, e2, e3, e4, e5, e6, e7, e8, e9]


def sub_lists(lst):
    """Yield successive n-sized chunks from lst."""
    n = 9
    return [lst[i:i + n] for i in range(0, len(lst), n)]

# Make List of all Cards

In [ ]:
card_fronts = []
card_backs = []

# Rule Cards - 4 copies
cards = 4*['final_cards/rule_cards/' + x.name for x in Path('final_cards/rule_cards/').glob('*.png')]
card_fronts += sorted(cards)
card_backs += ['final_cards/card_backs/BACK_rules.png' for c in cards]

# Telescope Cards
cards = ['final_cards/telescope_cards/' + x.name for x in Path('final_cards/telescope_cards/').glob('*.png')]
card_fronts += cards
card_backs += ['final_cards/card_backs/BACK_telescope.png' for c in cards]

# Observation Cards
cards = ['final_cards/obs_cards/' + x.name for x in Path('final_cards/obs_cards/').glob('*.png')]
card_fronts += cards
card_backs += ['final_cards/card_backs/BACK_observation.png' for c in cards]

# Science Goals
cards = ['final_cards/goal_cards/' + x.name for x in Path('final_cards/goal_cards/').glob('*.png')]
card_fronts += cards
card_backs += ['final_cards/card_backs/BACK_goal.png' for c in cards]

# Chaos Cards
cards = ['final_cards/chaos_cards/' + x.name for x in Path('final_cards/chaos_cards/').glob('*.png')]
card_fronts += cards
card_backs += ['final_cards/card_backs/BACK_chaos.png' for c in cards]



# Make Figures from the Cards

In [ ]:
# Split up into pages of 9
for i, cards in enumerate(sub_lists(card_fronts)):
    # CARD FRONTS
    fig, card_locs = setup_pdf()
    for i, card in enumerate(cards):
        img = Image.open(card).resize((1000,1400))
        plt.imshow(img, extent = card_locs[i])
    plt.tight_layout()
    front_pages.append(fig)
    plt.show()

for i, cards in enumerate(sub_lists(card_backs)):
    # CARD BACKS
    fig, card_locs = setup_pdf()
    e_reversed = [card_locs[2], card_locs[1], card_locs[0],
                card_locs[5], card_locs[4], card_locs[3],
                card_locs[8], card_locs[7], card_locs[6],
                ] # reversed order for card backs
    for i, card in enumerate(cards):
        img = Image.open(card).resize((1000,1400))
        plt.imshow(img, extent = e_reversed[i])

    plt.tight_layout()
    back_pages.append(fig)
    plt.show()

# all pages = alternate front and backs
for i in range(len(front_pages)):
    all_pages.append(front_pages[i])
    all_pages.append(back_pages[i])



# Save to one PDF
This takes a few minutes to render.

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages

# Collated PDF (front and backs on alternating pages)
with PdfPages('../printable_pdf/MAST-Match_Collated.pdf') as pdf:
    for fig in all_pages:
        pdf.savefig(fig, dpi=300)

# Card Fronts Only
with PdfPages('../printable_pdf/MAST-Match_CardFronts.pdf') as pdf:
    for fig in front_pages:
        pdf.savefig(fig, dpi=300)

# Card Backs Only
with PdfPages('../printable_pdf/MAST-Match_CardBacks.pdf') as pdf:
    for fig in back_pages:
        pdf.savefig(fig, dpi=300)


In [ ]:
print(f"Total of {len(card_fronts)} cards on {len(front_pages)} pages!")